In [ ]:
!pip install ytmusicapi
!ytmusicapi oauth


In [ ]:
import json
import random
import convert
from ytmusicapi import YTMusic
import pandas as pd

with open("YourLibrary.json", encoding="utf-8") as f:
    library = json.load(f)

with open("Playlist1.json", encoding="utf-8") as f:
    playlists = json.load(f)['playlists']

jsons = ["Streaming_History_Audio_2018-2020_0.json",
"Streaming_History_Audio_2020-2022_1.json",
"Streaming_History_Audio_2022-2023_2.json",
"Streaming_History_Audio_2023-2024_3.json"]
full_streaming_history = pd.concat(pd.read_json(json) for json in jsons)

ytmusic = YTMusic("browser.json")

# auto reload library
%load_ext autoreload
%autoreload 2

In [ ]:
artists = ytmusic.get_library_artists(limit=1000)

In [ ]:
[k for k in artists if "smith" in k['artist'].lower()]

# LIKED TRACKS

In [ ]:
liked_tracks = library['tracks']
playlist_data, uri_dict = convert.convert_to_playlist(ytmusic, {
    "name": "Liked Songs",
    "items": liked_tracks,
    "description": "Liked songs from Spotify"
}, from_liked_track_list=True, dry_run=True)

In [ ]:
with open("uri_dict.json", "w") as f:
    json.dump(uri_dict, f, indent=4)

In [ ]:
playlist_data = []
duplicate_uris = []
for k,v in uri_dict.items():
    if v not in playlist_data:
        playlist_data.append(v)
    else:
        duplicate_uris.append(k)

In [ ]:
liked_tracks = library['tracks']
playlist_data, uri_dict = convert.convert_to_playlist(ytmusic, {
    "name": "Spotify Liked Songs",
    "items": liked_tracks,
    "description": "Liked songs from Spotify"
}, wet_run_data=(playlist_data, uri_dict))

In [ ]:
# Check what stuff was left out because it was not found or was a duplicate

def find_track_name(uri):
    for track in liked_tracks:
        if track['uri'] == uri:
            return f"{track['track']} - {track['artist']} - {track['album']}"
    return None

a = {}
for k, v in uri_dict.items():
    if v not in a:
        a[v] = []
    a[v].append(k)

[[f"https://youtu.be/watch?v={i}", [find_track_name(j) for j in a[i]]] for i in a if len(a[i]) > 1]

# PLAYLISTS

In [24]:
len(playlists)

136

# ALBUMS

In [ ]:
albums = pd.DataFrame(library['albums'])

In [ ]:
album = library['albums'][53]
# album = library['albums'][25]


In [ ]:
album

In [ ]:
def verify_match(album, album_result):
    title_match = album['album'].lower() == album_result['title'].lower()
    artist_match = album['artist'].lower(
    ) == album_result['artists'][0]['name'].lower()

    if not title_match and not artist_match:
        return False
    if title_match and not artist_match:
        print(
            f"Title match but artist mismatch: {album['album']} {album['artist']}")
        substring_search = album['artist'].lower() in album_result['artists'][0]['name'].lower()
        if not substring_search:
            print(f"Substring search failed: {album['artist']} {album_result['artists'][0]['name']}")
            return False
        return True
    if not title_match and artist_match:
        print(
            f"Artist match but title mismatch: {album['album']} {album['artist']}")
        return True
    return True


search_query = f"{album['album']}"
if album['artist'] != "Various Artists":
    search_query += f" {album['artist']}"

results = ytmusic.search(
    search_query, filter="albums", ignore_spelling=True)

if len(results) == 0:
    print(f"Could not find {album['album']} {album['artist']}")


matches = [result for result in results if verify_match(album, result)]

if len(matches) == 0:

    print(f"results: {results}")
    raise Exception(f"Could not match {album['album']} {album['artist']}")

In [ ]:
matches[0]

In [ ]:
convert.search_for_album(ytmusic, album)

In [ ]:
from tqdm import tqdm
results = []
failed = []
for album in tqdm(library['albums']):
    try:
        res = convert.search_for_album(ytmusic, album)
        results.append(ytmusic.rate_playlist(res, "LIKE"))
    except:
        failed.append(album)

In [ ]:
[f"{album['album']} - {album['artist']}" for album in failed]

In [ ]:
len(library['albums'])
res = convert.search_for_album(ytmusic,library['albums'][2])
ytmusic.rate_playlist(res[0]["playlistId"], "LIKE")


# ARTISTS

In [ ]:
from tqdm import tqdm
from time import sleep
import convert

artists = pd.DataFrame(library["artists"])

artists['channelId'] = ""

max = 100
redo = []
for i, row in tqdm(artists.iterrows(), total=len(artists)):
    if row["channelId"] == "":
        try:
            artists.loc[i, 'channelId'] = convert.search_for_artist(ytmusic, row["name"])
        except:
            print(f"Error with {row['name']}")
            redo.append(i)
            
        if i % max == 0:
            tqdm.write(f"Sleeping for 15 seconds")
            sleep(5)

# artists.to_csv("artist_cids_uncorrected.csv", index=False)


In [ ]:
# artists = pd.read_csv("artist_cids_uncorrected.csv")

empty = []
fixed = []

artists['test_track'] = ""

for index, row in artists.iterrows():
    test_track = ""
    if row["name"] in full_streaming_history["master_metadata_album_artist_name"].values:
        test_track = full_streaming_history[full_streaming_history["master_metadata_album_artist_name"] == row["name"]].iloc[0]['master_metadata_track_name']
    artists.loc[index, 'test_track'] = test_track


def get_correct_id(index, row):
    try:
        return ytmusic.search(f"{row['name']} - {row['test_track']}", filter="songs", ignore_spelling=True)[0]["artists"][0]["id"]
    except:
        empty.append([index, ytmusic.search(f"{row['name']} - {row['test_track']}", filter="songs", ignore_spelling=True)])
        return row["channelId"]
    
for index, row in tqdm(artists.iterrows(), total=len(artists)):
    if row["test_track"] != "":
        correct_id = get_correct_id(index, row)
        if correct_id != row["channelId"]:
            fixed.append([
                index,
                row["name"],
                row["channelId"],
                correct_id
            ])
            artists.loc[index, 'channelId'] = correct_id

artists['alternateChannelId'] = ""

for index, artist_name, alt_id, new_id in fixed:
    if alt_id == artists.iloc[index]['channelId']:
        continue
    if new_id != artists.iloc[index]['channelId']:
        continue
    artists.loc[index, 'alternateChannelId'] = alt_id


In [ ]:
# check which artists have not been corrected
def channel_url(channel_id):
    return f"https://music.youtube.com/channel/{channel_id}"

[(k[0], f"{artists.iloc[k[0]]['name']} - {artists.iloc[k[0]]['test_track']}", channel_url(artists.iloc[k[0]]["channelId"])) for k in empty],
[(k[1], f"{channel_url(k[2])}",f"{channel_url(k[3])}") for k in fixed]

In [ ]:
# update subscribed status for artists
for channelId in tqdm(artists.channelId.values):
    if channelId == "" or pd.isna(channelId):
        continue
    artists.loc[artists['channelId'] == channelId,
                'subscribed'] = ytmusic.get_artist(channelId)['subscribed']

In [ ]:
#subscribe to artists
# needs channelId, subscribed, alternateChannelId, name for each artist

responses = []
for index, channelId in enumerate(tqdm(artists[artists['subscribed'] == False].channelId.values)):

    if channelId == "" or pd.isna(channelId) or artists.iloc[index]['subscribed'] == True:
        continue

    if ytmusic.get_artist(channelId)['subscribed']:
        continue

    responses.append([index, ytmusic.subscribe_artists(channelId)])

    sleep(2)
    if not ytmusic.get_artist(channelId)['subscribed']:
        alt_id = artists.iloc[index]['alternateChannelId']

        if alt_id != "":
            responses.append([index, ytmusic.subscribe_artists(alt_id)])

            sleep(2)
            if not ytmusic.get_artist(alt_id)['subscribed']:
                print(f"Failed to subscribe to {artists.iloc[index]['name']}"
                      )

# Reverse Search

In [ ]:
artist_name = 'Mount Kimbie'
f"{full_streaming_history[full_streaming_history['master_metadata_album_artist_name'] == artist_name]['ms_played'].sum() / 1000 / 60 / 60:2f} hours listening to {artist_name}"